In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from statsmodels.tsa.stattools import acf,pacf
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [6]:
south_east = pd.read_csv("Brazil_Data/southeast.csv")

In [7]:
max_date = south_east['Data'].max()
min_date = south_east['Data'].min()

stations = south_east['station_code'].unique()

print(min_date, max_date)
print(stations)

start_date = '2020-04-30'

2000-05-07 2021-04-30
['A601' 'A705' 'A507' 'A508' 'A706' 'A506' 'A602' 'A502' 'A505' 'A603'
 'A604' 'A708' 'A707' 'A509' 'A510' 'A520' 'A523' 'A701' 'A525' 'A609'
 'A512' 'A727' 'A511' 'A715' 'A522' 'A611' 'A612' 'A725' 'A728' 'A513'
 'A521' 'A524' 'A519' 'A608' 'A530' 'A711' 'A716' 'A531' 'A729' 'A617'
 'A606' 'A517' 'A528' 'A713' 'A607' 'A615' 'A714' 'A516' 'A538' 'A619'
 'A718' 'A514' 'A618' 'A726' 'A527' 'A712' 'A515' 'A610' 'A616' 'A614'
 'A545' 'A613' 'A621' 'A536' 'A534' 'A543' 'A533' 'A526' 'A542' 'A652'
 'A734' 'A735' 'A550' 'A551' 'A736' 'A547' 'A532' 'A739' 'A541' 'A554'
 'A529' 'A737' 'A740' 'A733' 'A518' 'A544' 'A539' 'A553' 'A546' 'A537'
 'A548' 'A540' 'A738' 'A552' 'A549' 'A623' 'A747' 'A741' 'A555' 'A620'
 'A535' 'A622' 'A746' 'A753' 'A624' 'A556' 'A748' 'A755' 'A657' 'A557'
 'F501' 'A561' 'A667' 'A659' 'S122' 'A563' 'A569' 'A626' 'A764' 'A625'
 'A762' 'A559' 'A565' 'A767' 'A631' 'A634' 'A633' 'A570' 'A765' 'A635'
 'A744' 'A568' 'A562' 'A766' 'A768' 'A564' 'A769' 'A567

In [11]:
renamed_columns_en = ['date','hour','total precipitation (mm)','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)','max. temperature in the previous hour (°c)','min. temperature in the previous hour (°c)','dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','relative humidity max. in the previous hour (%)','relative humidity min. in the previous hour (%)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)','wind speed (m/s)','region','state','station','station_code','latitude','longitude','height']

def process_raw(df):
    """
    ! drop index
    ! rename the columns
    ! combine hour and date columns -> transform to date_time
    ! append columns data from all stations to keep the data granularity
    ! remove not useful columns
    ! return table 
    """
    df.drop(['index'],inplace=True, axis=1)
    df.columns = renamed_columns_en
    df['date_time'] = pd.to_datetime(df['date'] + ' ' +  df['hour'])
    df.drop(['date','hour','pressao atmosferica ao nivel da estacao (mb)','atmospheric pressure max. in the previous hour (mb)','atmospheric pressure min. in the previous hour (mb)','radiation (kj/m2)','air temperature - dry bulb (°c)','dew point temperature (°c)', 'dew temperature max. in the previous hour (°c)','dew temperature min. in the previous hour (°c)','air relative humidity (%)','wind direction (° (gr))','wind rajada maxima (m/s)'], inplace=True, axis=1)

    # station_codes = list(df.station_code.unique())
    # df_models = by_code(df,station_codes[0])
    # if len(station_codes)>1:
    #     for i in station_codes[1:]:
    #         df_temp = by_code(df,i)
    #         df_models = pd.concat([df_models,df_temp],axis=1)
    return df

# def by_code(df, station_code):
#     """
#     ! function that filters the dataframe by station_code and rename the columns by the status code
#     """
#     df_station = None
#     df_station = df[df['station_code'] == station_code]
#     df_station.index = df_station.iloc[:,-1]
#     df_station.drop(['station_code','date_time'],axis=1,inplace=True)
#     df_station.columns = list(map(lambda x: station_code+'_'+x, df_station.columns))
    return df_station

def clean_na(df, na_value = -9999):
    """ 
    ! replace Na from dataset
    """
    df = df.replace(to_replace=na_value,value=np.NaN)
    df = df[df.iloc[:,1].first_valid_index():]
    df = df.fillna(method='ffill')
    return df

def make_dataset(stations, start_date, df):
    """
    ! filter raw data from date and stations code
    ! process data
    ! clean na
    """
    df = df[df['Data'] >= start_date]
    df = df[df['station_code'].apply(lambda x: x in stations)]
    df = process_raw(df)
    # df = clean_na(df)
    return df.reset_index()   


In [12]:
south_east_Clean = make_dataset(stations,'2019-04-03', south_east)
south_east_Clean

# the start_date variable I set up produces an empty table

,index,total precipitation (mm),max. temperature in the previous hour (°c),min. temperature in the previous hour (°c),relative humidity max. in the previous hour (%),relative humidity min. in the previous hour (%),wind speed (m/s),region,state,station,station_code,latitude,longitude,height,date_time


In [10]:
south_east_Clean.groupby(by=[south_east_Clean['date_time'].dt.year, 
    south_east_Clean['date_time'].dt.month,
    south_east_Clean['date_time'].dt.day ]).mean()
r

index  total precipitation (mm)  \
date_time date_time date_time                                           
2019      4         3          1.295935e+07               -663.171655   
                    4          1.295937e+07               -617.838832   
                    5          1.295940e+07               -623.439683   
                    6          1.295942e+07               -620.487358   
                    7          1.295954e+07               -634.588492   
...                                     ...                       ...   
2021      4         26         1.513152e+07              -2608.784172   
                    27         1.513155e+07              -2687.007662   
                    28         1.513157e+07              -2796.091163   
                    29         1.513159e+07              -2798.810515   
                    30         1.513162e+07              -2754.184732   

                               max. temperature in the previous hour (°c)  \
date_time date_time date_time                                               
2019      4         3                                         -103.603288   
                    4                                          -51.539116   
                    5                                          -56.330215   
                    6                                          -51.125113   
                    7                                          -65.861423   
...                                                                   ...   
2021      4         26                                       -2535.827852   
                    27                                       -2617.431236   
                    28                                       -2732.855593   
                    29                                       -2741.977069   
                    30                                       -2686.434508   

                               min. temperature in the previous hour (°c)  \
date_time date_time date_time                                               
2019      4         3                                         -104.949008   
                    4                                          -52.947421   
                    5                                          -57.720663   
                    6                                          -52.416440   
                    7                                          -67.083305   
...                                                                   ...   
2021      4         26                                       -2536.922008   
                    27                                       -2618.334760   
                    28                                       -2733.839625   
                    29                                       -2742.915772   
                    30                                       -2687.332075   

                               relative humidity max. in the previous hour (%)  \
date_time date_time date_time                                                    
2019      4         3                                              -266.371315   
                    4                                              -201.573696   
                    5                                              -265.128968   
                    6                                              -292.697846   
                    7                                              -388.802721   
...                                                                        ...   
2021      4         26                                            -2847.841723   
                    27                                            -2909.221756   
                    28                                            -3000.237416   
                    29                                            -2983.610738   
                    30                                            -2990.073546   

                 